# ONS Core

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [1]:
import os
from datetime import date, datetime, timedelta

import numpy as np

import unittest

import common_core

## Configuration

Data to download from the NHS statistical work area

In [2]:
# The latest ONS age bands
ageDemographics = [
    '<1', '1-4', '5-9', '10-14', '15-19', '20-24', '25-29',
    '30-34', '35-39', '40-44', '45-49', '50-54', '55-59',
    '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90+'
]

# The legacy ONS age bands
legacyAgeDemographics = [
    '01-14', '15-44', '45-64', '65-74', '75-84', '85+'
]

## Constants

Text strings to avoid hard-coded values throughout the code; avoids clutter and silent errors.

In [3]:
# Index from Fri 2 Jan 1970
minWeek = date(1970, 1, 2)
maxWeek = date(datetime.now().year, 12, 31)

In [4]:
# Weeks have an end date and a number, always ending on Fridays
WEEK_ENDED = "week_ended"
WEEK_NUMBER = "week_number"

# Internal names used by the cache
WEEK_COL_NOS = "week_col_nos"
WEEK_NUMBERS = "week_numbers"
WEEK_ENDINGS = "week_endings"
WEEK_OFFSETS = "week_offsets"

# Deaths are reported by registration date and occurrence date
TOTAL_REGISTRATIONS = "total_registrations"
TOTAL_OCCURRENCES = "total_occurrences"
COVID_REGISTRATIONS = "covid_registrations"
COVID_OCCURRENCES = "covid_occurrences"
COVID_UNDERLYING = "covid_underlying"

In [5]:
ONS_DEATHS = "ons-deaths"
ONS_LAHB_DEATHS = "ons-lahb-deaths"
ONS_EWM_DEATHS = "ons-ewm-deaths"

## Date Helper Functions

Functions to calculate week endings, etc.

In [6]:
def getFirstFriday(year):
    '''Get the first Friday of the year'''

    # Week 1 ends on/after Jan 2nd
    epoch = datetime.strptime(f"{year}-01-02", '%Y-%m-%d').date()
    while epoch.weekday() != 4:
        epoch += timedelta(days=1)

    return epoch


def getWeekEnded(year, weekNo):
    '''Get the Friday for specific week ended in a given year'''
    
    weekEnded = getFirstFriday(year)
    weekEnded += timedelta(weeks=weekNo - 1)
    
    return(weekEnded)

## Load CSV Files

Load CSV data into cache

In [7]:
def loadCsvFile(folder, period, areaType, areaName, verbose=common_core.verbose):
    '''Load weekly registrations and occurrences for a single area into cache'''

    csvPath = os.path.join(common_core.dataDir, folder, "csv", period, areaType)
    csvFn = os.path.join(csvPath, common_core.getSafeName(areaName) + ".csv")

    if os.path.exists(csvFn):
        data = common_core.loadCsvIntoArray(csvFn, verbose = verbose)
    else:
        data = None

    return data


def loadCsvFiles(folder, period, verbose=common_core.verbose):
    '''Load weekly registrations and occurrences for all areas into cache'''

    cache = {}

    for nationName in common_core.nationNames:
        data = loadCsvFile(folder, period, "nation", nationName, verbose = verbose)
        if data is not None:
            cache[nationName] = data

    for regionName in common_core.regionNames:
        data = loadCsvFile(folder, period, "region", regionName, verbose = verbose)
        if data is not None:
            cache[regionName] = data

    return cache

## NumPy Helper Functions

Useful functionality such as moving average or rolling sum

In [8]:
def shiftRegistrations(data):
    """Shift registration data left by half a period"""

    # Final value is invalid (so not included in the convolution result) and needs to be zero
    result = np.append(np.convolve(data, np.array([0.5, 0.5]), mode="valid"), 0)

    return result

In [9]:
class TestShiftRegistrations(unittest.TestCase):
    '''Class to test rollingSum function'''

    def testShift(self):
        '''Test processing of a list shorter than the window size'''

        actual = shiftRegistrations(np.arange(6))
        expected = np.array([0.5, 1.5, 2.5, 3.5, 4.5, 0])

        self.assertEqual((actual == expected).all(), True)

## Automated Testing

In [10]:
if __name__ == '__main__':

    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
